In [22]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install py7zr

In [29]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset

import json

In [44]:
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/SmolLM2-135M",                        # Change here for model (use unsloth models only: https://huggingface.co/unsloth)
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

==((====))==  Unsloth 2025.2.4: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [37]:
# Change according to your dataset

prompt = """Below is a conversation dialogue between people. Write a short summary for the conversation.

### Dialogue:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["dialogue"]
    # inputs       = examples["input"]
    outputs      = examples["summary"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = load_dataset("Samsung/samsum",  trust_remote_code=True)
dataset_train = dataset['train']
dataset_eval = dataset['validation']
dataset_test = dataset['test']
dataset_test = dataset_test.select(range(100))
dataset_samsum = dataset_train.map(formatting_prompts_func, batched = True)
dataset_eval = dataset_eval.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [38]:
# Change according to your dataset

def get_inference(dialog):
  FastLanguageModel.for_inference(model)
  inputs = tokenizer(
  [
      prompt.format(
          dialog,
          "", # output
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
  return tokenizer.batch_decode(outputs)[0].split("Response:\n")[-1]

In [39]:
# Generate responses using map (Inference before fine-tuning)
# before_training = list(map(get_inference, (entry["dialogue"] for entry in dataset_test)))


# with open("before_training.json", "w", encoding="utf-8") as f:
#     json.dump(before_training, f, ensure_ascii=False, indent=4)

In [45]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_samsum,
    eval_dataset = dataset_eval,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        # gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 500,
        # learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        eval_strategy="steps",
        eval_steps = 100,
        optim = "adamw_8bit",
        # weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"
        ),)


#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

trainer_stats = trainer.train()

GPU = Tesla T4. Max memory = 14.741 GB.
4.002 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 14,732 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 1
\        /    Total batch size = 2 | Total steps = 500
 "-____-"     Number of trainable parameters = 4,884,480


Step,Training Loss,Validation Loss
100,2.683500,2.590220
200,2.204500,2.319030
300,2.109800,2.289377
400,2.029800,2.277446
500,2.227900,2.273768


In [46]:
get_inference(dataset_test[0]["dialogue"])

"Hannah has Betty's number.<|endoftext|>"

In [47]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

314.993 seconds used for training.
5.25 minutes used for training.
Peak reserved memory = 4.002 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 27.149 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [48]:
after_training = [get_inference(entry["dialogue"]) for entry in dataset_test]
with open("after_training.json", "w", encoding="utf-8") as f:
    json.dump(after_training, f, ensure_ascii=False, indent=4)

Evaluation

In [49]:
%%capture
!pip install evaluate
!pip install rouge_score

In [50]:
import evaluate
rouge = evaluate.load('rouge')
bleu = evaluate.load("bleu")

In [51]:
rouge.compute(predictions=before_train, references=dataset_test['summary'])

NameError: name 'before_train' is not defined

In [ ]:
rouge.compute(predictions=after_train, references=dataset_test['summary'])

In [ ]:
bleu.compute(predictions=before_train, references=dataset_test['summary'])

In [ ]:
bleu.compute(predictions=after_train, references=dataset_test['summary'])

In [ ]:
dataset_test[0]

In [ ]:
glue = evaluate.load("glue", "sst2")

In [ ]:
glue.compute(predictions=before_train, references=dataset_test['summary'])

In [ ]:
glue.compute(predictions=dataset_test['summary'], references=dataset_test['summary'])